In [19]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from imblearn.over_sampling import SMOTE
import numpy as np
#import jplus



In [20]:
#load databases

#DataDir = './data/'
DataDir = '.'
import pickle
dset = pickle.load(open('%s/training_full.data'%DataDir))
print dset.keys()


featnames = ['dm_j0660','J0378','J0395','J0410','J0430','J0515',
             'J0660','J0861','uSDSS','gSDSS','rSDSS','iSDSS','zSDSS']

ntrain = len(dset['obj'])
feat_arr = []

import itertools

terms = featnames[1:] # create all colour combinations
nterms = len(terms)
ncomb = int(nterms*(nterms-1)/2.)
print 'All colours:', ncomb
comb = list(itertools.combinations(terms,2))
lcomb = list(comb)
colournames = ['%s - %s'%(x[0], x[1]) for x in list(comb)]

for x in range(ntrain):
    fx = []
    for y in featnames:
        fx.append(dset[y][x,0])
    for z in range(ncomb):
        fx.append(dset[lcomb[z][0]][x,0] - dset[lcomb[z][1]][x,0])
        
#    fx.append(dset['uSDSS'][x,0]-dset['gSDSS'][x,0])
#    fx.append(dset['gSDSS'][x,0]-dset['rSDSS'][x,0])
#    fx.append(dset['rSDSS'][x,0]-dset['iSDSS'][x,0])
#    fx.append(dset['uSDSS'][x,0]-dset['gSDSS'][x,0])
#    fx.append(dset['iSDSS'][x,0]-dset['zSDSS'][x,0])
#    fx.append(dset['uSDSS'][x,0]-dset['zSDSS'][x,0])
#    fx.append(dset['rSDSS'][x,0]-dset['zSDSS'][x,0])
    
    feat_arr.append(fx)
class_arr = dset['class']
featnames += colournames
print featnames

['rSDSS', 'iSDSS', 'obj', 'gSDSS', 'J0395', 'zSDSS', 'J0378', 'J0430', 'uSDSS', 'dm_j0660', 'J0660', 'J0410', 'J0515', 'J0861', 'class']
All colours: 66
['dm_j0660', 'J0378', 'J0395', 'J0410', 'J0430', 'J0515', 'J0660', 'J0861', 'uSDSS', 'gSDSS', 'rSDSS', 'iSDSS', 'zSDSS', 'J0378 - J0395', 'J0378 - J0410', 'J0378 - J0430', 'J0378 - J0515', 'J0378 - J0660', 'J0378 - J0861', 'J0378 - uSDSS', 'J0378 - gSDSS', 'J0378 - rSDSS', 'J0378 - iSDSS', 'J0378 - zSDSS', 'J0395 - J0410', 'J0395 - J0430', 'J0395 - J0515', 'J0395 - J0660', 'J0395 - J0861', 'J0395 - uSDSS', 'J0395 - gSDSS', 'J0395 - rSDSS', 'J0395 - iSDSS', 'J0395 - zSDSS', 'J0410 - J0430', 'J0410 - J0515', 'J0410 - J0660', 'J0410 - J0861', 'J0410 - uSDSS', 'J0410 - gSDSS', 'J0410 - rSDSS', 'J0410 - iSDSS', 'J0410 - zSDSS', 'J0430 - J0515', 'J0430 - J0660', 'J0430 - J0861', 'J0430 - uSDSS', 'J0430 - gSDSS', 'J0430 - rSDSS', 'J0430 - iSDSS', 'J0430 - zSDSS', 'J0515 - J0660', 'J0515 - J0861', 'J0515 - uSDSS', 'J0515 - gSDSS', 'J0515 - rSD

In [7]:
print featnames, len(featnames)



['dm_j0660', 'J0378', 'J0395', 'J0410', 'J0430', 'J0515', 'J0660', 'J0861', 'uSDSS', 'gSDSS', 'rSDSS', 'iSDSS', 'zSDSS', 'J0378 - J0395', 'J0378 - J0410', 'J0378 - J0430', 'J0378 - J0515', 'J0378 - J0660', 'J0378 - J0861', 'J0378 - uSDSS', 'J0378 - gSDSS', 'J0378 - rSDSS', 'J0378 - iSDSS', 'J0378 - zSDSS', 'J0395 - J0410', 'J0395 - J0430', 'J0395 - J0515', 'J0395 - J0660', 'J0395 - J0861', 'J0395 - uSDSS', 'J0395 - gSDSS', 'J0395 - rSDSS', 'J0395 - iSDSS', 'J0395 - zSDSS', 'J0410 - J0430', 'J0410 - J0515', 'J0410 - J0660', 'J0410 - J0861', 'J0410 - uSDSS', 'J0410 - gSDSS', 'J0410 - rSDSS', 'J0410 - iSDSS', 'J0410 - zSDSS', 'J0430 - J0515', 'J0430 - J0660', 'J0430 - J0861', 'J0430 - uSDSS', 'J0430 - gSDSS', 'J0430 - rSDSS', 'J0430 - iSDSS', 'J0430 - zSDSS', 'J0515 - J0660', 'J0515 - J0861', 'J0515 - uSDSS', 'J0515 - gSDSS', 'J0515 - rSDSS', 'J0515 - iSDSS', 'J0515 - zSDSS', 'J0660 - J0861', 'J0660 - uSDSS', 'J0660 - gSDSS', 'J0660 - rSDSS', 'J0660 - iSDSS', 'J0660 - zSDSS', 'J0861 - uSD

In [8]:
#Example sets

def class_to_int(istr):
    if istr == 'Halpha':
        return 0
    elif istr == 'OIII+Hbeta':
        return 1
    if istr == 'OII':
        return 2
    elif istr == 'contaminant':
        return 3
    else:
        print '%s not recognised'%istr
        return -99


y_train_int = [class_to_int(x) for x in class_arr]
y_train = keras.utils.to_categorical(y_train_int, num_classes=len(np.unique(y_train_int)))
x_train = feat_arr
nfeat = len(feat_arr[0])
print nfeat
print y_train_int



79
[0, 2, 2, 0, 0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 2, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

Scale data using scikit-learn StandardScaler

In [9]:
from sklearn.preprocessing import StandardScaler 
Scaledata = True

if Scaledata:
    print 'scaling data...',
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    #x_test  = scaler.transform(x_test)
    print 'done'


scaling data... done


In [10]:
# Balance the training set by sub-sampling the contaminant class

all_classes = [0,1,2,3]
nclass = []
selclass = []
yarr = np.asarray(y_train_int)
for ic in all_classes:
    sel = np.where(np.asarray(yarr) == ic)[0]
    nclass.append(len(yarr[sel]))
    selclass.append(sel)


idd = list(np.random.choice(selclass[3], nclass[2]))
print idd, yarr[idd]
nff = sum(nclass)

new_idd = []
for ic in [0,1,2]:
    sel = np.where(yarr == ic)[0]
    new_idd += list(sel)

new_idd += idd
        
y_new = np.asarray([y_train[x] for x in new_idd])
x_new = np.asarray([x_train[x] for x in new_idd])



[338, 561, 493, 324, 352, 215, 221, 240, 225, 124, 364, 506, 101, 474, 624, 532, 271, 532, 365, 190, 205, 331, 615, 75, 205, 678, 364, 338, 652, 89, 237, 224, 386, 246, 57, 38, 626, 282, 649, 130, 252, 182, 507, 468, 37, 441, 352, 177, 434, 174, 623, 458, 528, 84, 138] [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [11]:

print y_new

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1.

In [13]:
# Keras MLP

from keras import metrics

model = Sequential()
model.add(Dense(22, activation='softmax', input_dim=nfeat))
model.add(Dense(22, activation='softmax'))
model.add(Dense(4, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='adam')

model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 22)                1760      
_________________________________________________________________
dense_5 (Dense)              (None, 22)                506       
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 92        
Total params: 2,358
Trainable params: 2,358
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(x_new, y_new,
          shuffle=True,
          verbose=1,
          epochs=100
          )



Epoch 1/100
140/140 [==============================] - 3s 18ms/step - loss: 1.3697
Epoch 2/100
140/140 [==============================] - 0s 365us/step - loss: 1.3638
Epoch 3/100
140/140 [==============================] - 0s 489us/step - loss: 1.3578
Epoch 4/100
140/140 [==============================] - 0s 257us/step - loss: 1.3525
Epoch 5/100
140/140 [==============================] - 0s 258us/step - loss: 1.3467
Epoch 6/100
140/140 [==============================] - 0s 234us/step - loss: 1.3411
Epoch 7/100
140/140 [==============================] - 0s 280us/step - loss: 1.3358
Epoch 8/100
140/140 [==============================] - 0s 284us/step - loss: 1.3303
Epoch 9/100
140/140 [==============================] - 0s 228us/step - loss: 1.3254
Epoch 10/100
140/140 [==============================] - 0s 254us/step - loss: 1.3202
Epoch 11/100
140/140 [==============================] - 0s 201us/step - loss: 1.3147
Epoch 12/100
140/140 [==============================] - 0s 302us/step - los

140/140 [==============================] - 0s 402us/step - loss: 1.0034
Epoch 98/100
140/140 [==============================] - 0s 300us/step - loss: 0.9992
Epoch 99/100
140/140 [==============================] - 0s 364us/step - loss: 0.9947
Epoch 100/100
140/140 [==============================] - 0s 311us/step - loss: 0.9907


In [15]:
pred = model.predict(x_new,verbose=1)
pred_class = [np.argmax(x) for x in pred]
y_new_num = [np.argmax(x) for x in y_new]



140/140 [==============================] - 0s 1ms/step


In [16]:
from sklearn.metrics import f1_score
score = f1_score(y_new_num, pred_class,average=None)
print score
print np.unique(pred_class)

[0.         0.         0.97345133 0.78832117]
[2 3]


/Users/aaorsi/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
print pred_class
print 

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3]

